In [2]:
import numpy as np
import sys
import cv2
import torch as th
import copy
import blobfile as bf
import PIL
import json
import torchvision
sys.path.insert(1, '/home/mint/guided-diffusion/sample_scripts/sample_utils/')
import mani_utils, inference_utils, params_utils, vis_utils
sys.path.insert(1, '/home/mint/guided-diffusion/guided_diffusion/dataloader/')

cond, _ = params_utils.load_params(path='/data/mint/DPM_Dataset/ffhq_256_with_anno/params/valid/', params_key=['shadow'])


Key=> shadow : Filename=>/data/mint/DPM_Dataset/ffhq_256_with_anno/params/valid/ffhq-valid-shadow-anno.txt


In [15]:
# Appendix - figure
img_dict = {
    '60020.jpg':{'min_f':5, 'max_f':9},
    '60079.jpg':{'min_f':5, 'max_f':6},
    '60161.jpg':{'min_f':6, 'max_f':9},
    '60169.jpg':{'min_f':5, 'max_f':8},
    '60240.jpg':{'min_f':7, 'max_f':5},
    '60284.jpg':{'min_f':6, 'max_f':9},
    '60334.jpg':{'min_f':4, 'max_f':8},
    '60365.jpg':{'min_f':6, 'max_f':7},
    '60459.jpg':{'min_f':6, 'max_f':9},
    '60848.jpg':{'min_f':6, 'max_f':8},
}
json_out = {"pair" : {}}
n_frames = 15

# "pair": {
#     "pair1": {
#         "src": "63307.jpg",
#         "dst": "60000.jpg"
#     },
    
for i, (k, v) in enumerate(img_dict.items()):
    # Remove shadow
    rmv_step = v['min_f']
    rmv_step_size = np.linspace(cond[k]['shadow'] - 15, cond[k]['shadow'], num=n_frames)
    rmv_step_size = - np.abs(rmv_step_size[1:] - rmv_step_size[:-1])[0]
    rmv_bound = cond[k]['shadow'] + (rmv_step * rmv_step_size)
    
    # Add shadow
    add_step = v['max_f']
    add_step_size = np.linspace(cond[k]['shadow'], cond[k]['shadow'] + 5, num=n_frames)
    add_step_size = (add_step_size[1:] - add_step_size[:-1])[0]
    add_bound = cond[k]['shadow'] + (add_step * add_step_size)
    
    print(f"{k}: Shadow = {cond[k]['shadow']}")
    print(f"\t Bound = {cond[k]['shadow']-15} <-> {cond[k]['shadow']+5}")
    print(f"[#] Add shadow : step_size = {add_step_size}, bound at {add_bound}")
    print(f"[#] Remove shadow : step_size = {rmv_step_size}, bound at {rmv_bound}")
    print("\n")
    
    json_out["pair"][f"pair{i+1}"] = {
        "src" : k,
        "dst" : "60000.jpg",
        "rmv_shadow_bound" : rmv_bound[0],
        "add_shadow_bound" : add_bound[0],
    }

with open(f'/home/mint/guided-diffusion/sample_scripts/paper_script/sample_json/website/shadow_app_fig.json', 'w') as fp:
    json.dump(json_out, fp, indent=4)

60020.jpg: Shadow = [3.597798]
	 Bound = [-11.402202] <-> [8.597798]
[#] Add shadow : step_size = [0.35714286], bound at [6.81208372]
[#] Remove shadow : step_size = [-1.07142857], bound at [-1.75934485]


60079.jpg: Shadow = [0.30794513]
	 Bound = [-14.69205487] <-> [5.30794513]
[#] Add shadow : step_size = [0.35714286], bound at [2.45080227]
[#] Remove shadow : step_size = [-1.07142857], bound at [-5.04919773]


60161.jpg: Shadow = [2.95058346]
	 Bound = [-12.04941654] <-> [7.95058346]
[#] Add shadow : step_size = [0.35714286], bound at [6.16486917]
[#] Remove shadow : step_size = [-1.07142857], bound at [-3.47798797]


60169.jpg: Shadow = [1.93743204]
	 Bound = [-13.06256796] <-> [6.93743204]
[#] Add shadow : step_size = [0.35714286], bound at [4.79457489]
[#] Remove shadow : step_size = [-1.07142857], bound at [-3.41971082]


60240.jpg: Shadow = [2.80623041]
	 Bound = [-12.19376959] <-> [7.80623041]
[#] Add shadow : step_size = [0.35714286], bound at [4.59194469]
[#] Remove shadow 

In [14]:
# Appendix - Video 
img_name = [66248, 66646, 69398, 65169, 63307, 65401, 60922, 63648]

img_dict = {
    '66248.jpg':{'min_f':19, 'max_f':54},
    '66646.jpg':{'min_f':19, 'max_f':54},
    '69398.jpg':{'min_f':23, 'max_f':50},
    '65169.jpg':{'min_f':19, 'max_f':51},
    '63307.jpg':{'min_f':17, 'max_f':56},
    '65401.jpg':{'min_f':19, 'max_f':53},
    '60922.jpg':{'min_f':23, 'max_f':56},
    '63648.jpg':{'min_f':17, 'max_f':53},
}
n_frames = 30

json_out = {"pair":{}}
for i, (k, v) in enumerate(img_dict.items()):
    # Remove shadow
    rmv_step = n_frames - v['min_f']
    rmv_step_size = np.linspace(cond[k]['shadow'] - 15, cond[k]['shadow'], num=n_frames)
    rmv_step_size = - np.abs(rmv_step_size[1:] - rmv_step_size[:-1])[0]
    rmv_bound = cond[k]['shadow'] + (rmv_step * rmv_step_size)
    
    # Add shadow
    add_step = v['max_f'] - n_frames
    add_step_size = np.linspace(cond[k]['shadow'], cond[k]['shadow'] + 5, num=n_frames)
    add_step_size = (add_step_size[1:] - add_step_size[:-1])[0]
    add_bound = cond[k]['shadow'] + (add_step * add_step_size)
    
    print(f"{k}: Shadow = {cond[k]['shadow']}")
    print(f"\t Bound = {cond[k]['shadow']-15} <-> {cond[k]['shadow']+5}")
    print(f"[#] Add shadow : step_size = {add_step_size}, bound at {add_bound}")
    print(f"[#] Remove shadow : step_size = {rmv_step_size}, bound at {rmv_bound}")
    print("\n")
    
    json_out["pair"][f"pair{i+1}"] = {
        "src" : k,
        "dst" : "60000.jpg",
        "rmv_shadow_bound" : rmv_bound[0],
        "add_shadow_bound" : add_bound[0],
    }

with open(f'/home/mint/guided-diffusion/sample_scripts/paper_script/sample_json/website/shadow_app_vid.json', 'w') as fp:
    json.dump(json_out, fp, indent=4)

66248.jpg: Shadow = [0.6479205]
	 Bound = [-14.3520795] <-> [5.6479205]
[#] Add shadow : step_size = [0.17241379], bound at [4.78585153]
[#] Remove shadow : step_size = [-0.51724138], bound at [-5.04173468]


66646.jpg: Shadow = [2.17976133]
	 Bound = [-12.82023867] <-> [7.17976133]
[#] Add shadow : step_size = [0.17241379], bound at [6.31769237]
[#] Remove shadow : step_size = [-0.51724138], bound at [-3.50989384]


69398.jpg: Shadow = [1.25687526]
	 Bound = [-13.74312474] <-> [6.25687526]
[#] Add shadow : step_size = [0.17241379], bound at [4.70515113]
[#] Remove shadow : step_size = [-0.51724138], bound at [-2.36381439]


65169.jpg: Shadow = [5.03402101]
	 Bound = [-9.96597899] <-> [10.03402101]
[#] Add shadow : step_size = [0.17241379], bound at [8.65471067]
[#] Remove shadow : step_size = [-0.51724138], bound at [-0.65563416]


63307.jpg: Shadow = [0.80104213]
	 Bound = [-14.19895787] <-> [5.80104213]
[#] Add shadow : step_size = [0.17241379], bound at [5.28380075]
[#] Remove shad